In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import joblib
import numpy as np

In [2]:
# Load dữ liệu gốc
original_data_path = 'updated_student_data.csv'
data = pd.read_csv(original_data_path)


In [3]:
# Kiểm tra thông tin cơ bản
print("Kích thước dữ liệu:", data.shape)
print("Các cột trong dữ liệu:", data.columns.tolist())
print("Số lượng giá trị thiếu:", data.isnull().sum())

Kích thước dữ liệu: (8231, 51)
Các cột trong dữ liệu: ['mssv', 'namsinh', 'gioitinh', 'noisinh', 'lopsh', 'khoa', 'hedt', 'khoahoc', 'chuyennganh2', 'diachi_tinhtp', 'xeploai', 'soquyetdinh', 'ngaycapvb', 'drl_hk1', 'drl_hk2', 'drl_hk3', 'drl_hk4', 'drl_hk5', 'drl_hk6', 'drl_hk7', 'drl_hk8', 'drl_hk9', 'drl_hk10', 'diemrl_TB', 'dtb_hk1', 'dtb_hk2', 'dtb_hk3', 'dtb_hk4', 'dtb_hk5', 'dtb_hk6', 'dtb_hk7', 'dtb_hk8', 'dtb_hk9', 'dtb_hk10', 'chungchi_av', 'dtb_toankhoa', 'dtb_tichluy', 'sotc_tichluy', 'sotc_hk1', 'sotc_hk2', 'sotc_hk3', 'sotc_hk4', 'sotc_hk5', 'sotc_hk6', 'sotc_hk7', 'sotc_hk8', 'sotc_hk9', 'sotc_hk10', 'sotc_daura', 'hocky_thu', 'label']
Số lượng giá trị thiếu: mssv                0
namsinh             0
gioitinh            0
noisinh             0
lopsh               0
khoa                0
hedt                0
khoahoc             0
chuyennganh2        0
diachi_tinhtp       0
xeploai          6400
soquyetdinh      6400
ngaycapvb        6400
drl_hk1             0
drl_hk2  

In [4]:
# Chia dữ liệu thành train (80%) và test (20%)
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42, shuffle=True)

# Lưu tập train thành file
train_data_path = 'train_data.csv'
train_data.to_csv(train_data_path, index=False)

In [5]:
# Chia tập test thành 8 file dựa trên các kỳ
hoc_ky_columns = [f'sotc_hk{i}' for i in range(1, 9)]

In [6]:
def split_test_by_hoc_ky_with_selected_features(test_data, hoc_ky_columns):
    test_files = {}

    # Duyệt qua các nhóm học kỳ từ 1 đến 8
    for i in range(1, 9):
        # Định nghĩa các cột cơ bản cần giữ lại
        base_columns = [
            'drl_hk1', 'dtb_hk1', 'sotc_hk1', 'diemrl_TB', 'chungchi_av', 'dtb_toankhoa', 'dtb_tichluy',
            'sotc_tichluy', 'sotc_daura', 'hocky_thu', 'namsinh', 'gioitinh', 'noisinh', 'lopsh', 'khoa',
            'hedt', 'khoahoc', 'chuyennganh2', 'diachi_tinhtp', 'xeploai', 'soquyetdinh', 'ngaycapvb', 'label'
        ]

        # Thêm các cột cho các học kỳ từ 2 đến học kỳ hiện tại
        for hk in range(2, i + 1):
            base_columns.extend([f'drl_hk{hk}', f'dtb_hk{hk}', f'sotc_hk{hk}'])

        # Step 1: Lọc dữ liệu dựa trên chỉ số và học kỳ
        filtered_rows = test_data[test_data['hocky_thu'] >= i]

        # Step 2: Chọn các cột cần thiết theo thứ tự đã định nghĩa trong base_columns
        required_columns = [col for col in test_data.columns if col in base_columns]
        hoc_ky_data = filtered_rows[required_columns].copy()

        # Lưu dữ liệu vào file CSV
        file_path = f'group_test_hk{i}_data.csv'
        hoc_ky_data.to_csv(file_path, index=False)

        # Lưu đường dẫn của file vào từ điển
        test_files[f'hoc_ky_{i}'] = file_path

    return test_files

# Gọi hàm chia dữ liệu test theo các học kỳ
hoc_ky_test_files = split_test_by_hoc_ky_with_selected_features(test_data, hoc_ky_columns)

In [7]:
# Tách X và y từ tập train
columns_to_drop = ['mssv', 'soquyetdinh', 'ngaycapvb', 'xeploai']
train_data_cleaned = train_data.drop(columns=columns_to_drop).copy()
train_data_cleaned = train_data_cleaned.applymap(lambda x: pd.NA if x == -1 else x)
X = train_data_cleaned.drop(columns=['label'])
y = train_data_cleaned['label']

# Mã hóa các biến phân loại
X_encoded = pd.get_dummies(X, drop_first=True)
feature_columns = X_encoded.columns

<ipython-input-7-6f8ccea48f2c>:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  train_data_cleaned = train_data_cleaned.applymap(lambda x: pd.NA if x == -1 else x)


In [8]:
from imblearn.over_sampling import ADASYN

In [9]:
# Chia dữ liệu thành train và validation
X_train, X_val, y_train, y_val = train_test_split(X_encoded, y, test_size=0.2, random_state=42, stratify=y)

# Áp dụng ADASYN cho dữ liệu huấn luyện
adasyn = ADASYN(sampling_strategy='minority', random_state=42)
X_train_resampled, y_train_resampled = adasyn.fit_resample(X_train, y_train)

print("Kích thước dữ liệu sau ADASYN:")
print("X_train_resampled:", X_train_resampled.shape)
print("y_train_resampled:", y_train_resampled.shape)

Kích thước dữ liệu sau ADASYN:
X_train_resampled: (6640, 8794)
y_train_resampled: (6640,)


In [10]:
print(pd.Series(y_train_resampled).value_counts())

label
0    3329
1    3311
Name: count, dtype: int64


In [11]:
# Các mô hình để huấn luyện
models = {
    'SVM': SVC(kernel='rbf', probability=True),
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42)
}
# Huấn luyện và đánh giá các mô hình
results = {}
saved_models = {}
for name, model in models.items():
    model.fit(X_train_resampled, y_train_resampled)  # Huấn luyện với dữ liệu sau SMOTE
    y_pred = model.predict(X_val)
    accuracy = accuracy_score(y_val, y_pred)
    precision = precision_score(y_val, y_pred, average='weighted')
    recall = recall_score(y_val, y_pred, average='weighted')
    f1 = f1_score(y_val, y_pred, average='weighted')
    confusion = confusion_matrix(y_val, y_pred)
    results[name] = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'confusion_matrix': confusion
    }
    # Lưu mô hình đã huấn luyện
    model_filename = f'model_{name.replace(" ", "_").lower()}.joblib'
    joblib.dump(model, model_filename)
    saved_models[name] = model_filename
    print(f"Model '{name}' saved to {model_filename}")

# Đánh giá trên tập test
def process_test_file(file_path, models, feature_columns, summary_results):
    test_data = pd.read_csv(file_path)
    if 'label' not in test_data.columns:
        print(f"File {file_path} does not contain 'label' column. Skipping...")
        return

    test_data_cleaned = test_data.applymap(lambda x: pd.NA if x == -1 else x)
    X_test = test_data_cleaned.drop(columns=['label'])
    y_test = test_data_cleaned['label']
    X_test_encoded = pd.get_dummies(X_test, drop_first=True)

    # Đồng bộ hóa các cột giữa train và test
    for col in feature_columns:
        if col not in X_test_encoded.columns:
            X_test_encoded[col] = 0
    X_test_encoded = X_test_encoded[feature_columns]  # Sắp xếp lại thứ tự cột

    for name, model in models.items():
        y_pred_test = model.predict(X_test_encoded)
        accuracy = accuracy_score(y_test, y_pred_test)
        precision = precision_score(y_test, y_pred_test, average='weighted')
        recall = recall_score(y_test, y_pred_test, average='weighted')
        f1 = f1_score(y_test, y_pred_test, average='weighted')
        confusion = confusion_matrix(y_test, y_pred_test)

        # Lưu kết quả đánh giá vào summary_results
        if name not in summary_results:
            summary_results[name] = []
        summary_results[name].append({
            'file': file_path,
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1_score': f1,
            'confusion_matrix': confusion
        })

        print(f"Model: {name} | File: {file_path}")
        print(f"Accuracy: {accuracy:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1-Score: {f1:.4f}")

        # Vẽ confusion matrix
        plt.figure(figsize=(8, 6))
        sns.heatmap(confusion, annot=True, fmt='d', cmap='Blues', xticklabels=["Not Graduated", "Graduated"], yticklabels=["Not Graduated", "Graduated"])
        plt.title(f"Confusion Matrix: {name} | File: {file_path}")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.show()

Model 'SVM' saved to model_svm.joblib


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Model 'Logistic Regression' saved to model_logistic_regression.joblib
Model 'Decision Tree' saved to model_decision_tree.joblib
Model 'Random Forest' saved to model_random_forest.joblib
Model 'Gradient Boosting' saved to model_gradient_boosting.joblib


In [12]:
# Tổng kết kết quả qua các file học kỳ
summary_results = {}
for hoc_ky, test_file_path in hoc_ky_test_files.items():
    process_test_file(test_file_path, models, feature_columns, summary_results)

# Tổng hợp kết quả vào DataFrame và lưu file CSV
final_results = []
detailed_results = []

for model_name, results in summary_results.items():
    model_summary = {
        'Model': model_name,
        'Avg_Accuracy': sum([r['accuracy'] for r in results]) / len(results),
        'Avg_Precision': sum([r['precision'] for r in results]) / len(results),
        'Avg_Recall': sum([r['recall'] for r in results]) / len(results),
        'Avg_F1_Score': sum([r['f1_score'] for r in results]) / len(results)
    }
    final_results.append(model_summary)

    for r in results:
        detailed_results.append({
            'Model': model_name,
            'File': r['file'],
            'Accuracy': r['accuracy'],
            'Precision': r['precision'],
            'Recall': r['recall'],
            'F1_Score': r['f1_score']
        })

# Lưu kết quả trung bình
final_results_df = pd.DataFrame(final_results)
final_results_df.to_csv('model_summary_results.csv', index=False)

# Lưu kết quả chi tiết
detailed_results_df = pd.DataFrame(detailed_results)
detailed_results_df.to_csv('detailed_model_results.csv', index=False)

print("Final summary saved to model_summary_results.csv")
print("Detailed results saved to detailed_model_results.csv")

Output hidden; open in https://colab.research.google.com to view.

In [13]:
# Tổng kết kết quả qua các file học kỳ
summary_results = {}
for hoc_ky, test_file_path in hoc_ky_test_files.items():

SyntaxError: incomplete input (<ipython-input-13-697e6f05ae41>, line 3)